In [1]:
import torch
from vocgan_generator import Generator
import soundfile as sf

In [2]:
torch.__version__

'1.9.1+cu111'

In [3]:
def get_vocgan(generator_path):
    generator = Generator(80, 4,
          ratios=[4, 4, 2, 2, 2, 2], mult=256,
          out_band=1)
    generator_checkpoint = torch.load(generator_path, map_location=torch.device('cpu'))
    generator.load_state_dict(generator_checkpoint['model_g'])
    return generator

In [4]:
generator_path = '/home/x1113228/personal/gitRepo/data/tacotron2/checkpoints/kss_29de09d_4500.pt'

In [5]:
generator = get_vocgan(generator_path)
generator.eval()
#generator = generator.cuda()

In [6]:
import numpy as np
import torch

from hparams import defaults
from model import Tacotron2
from train import load_model
from tokenizer import TacotronTokenizer

In [7]:
class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

In [8]:
hparams = Struct(**defaults)

In [10]:
checkpoint_path = "output_ver1"
tokenizer = TacotronTokenizer.from_pretrained(checkpoint_path)

In [11]:
checkpoint_path = "output_ver1/checkpoint_80000"
model = load_model(hparams, torch.device('cpu'))
model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu'))['state_dict'])
model.eval()
#model.to(torch.device('cuda'))

Tacotron2(
  (embedding): Embedding(80, 512)
  (encoder): Encoder(
    (convolutions): ModuleList(
      (0): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (lstm): LSTM(512, 256, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (prenet): Prenet(
      (layers): ModuleList(
        (0): LinearNorm(
          (line

In [18]:
text = "이거 진짜 잘 되는거 맞아?"
sequence = torch.tensor(tokenizer(text)).unsqueeze(0)

In [19]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)

Warning! Reached max decoder steps


In [20]:
audio = generator.generate_audio(mel_outputs_postnet)

In [21]:
sample_audio_path = 'sample_audio1.wav'
sf.write(sample_audio_path, audio, hparams.sampling_rate, 'PCM_24')

In [22]:
import IPython

In [23]:
IPython.display.Audio(sample_audio_path)

In [21]:
from g2pk import G2p

In [22]:
g2p = G2p()

In [23]:
g2p(text)

'이거 진짜 잘 되는거 마즘?'